In [41]:
%matplotlib inline
import torch
import numpy as np
import requests
from PIL import Image
import io
from torch.autograd import Variable

In [42]:
np.random.seed(0)

In [43]:
from torchvision import models, transforms

In [44]:
net = models.vgg16(pretrained=True)

In [45]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Scale(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   normalize
])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [75]:
response = requests.get('https://s3.amazonaws.com/outcome-blog/wp-content/uploads/2017/02/25192225/cat.jpg')
img_pil = Image.open(io.BytesIO(response.content))
img_pil1 = Image.open('./resources/guitar.png')

In [76]:
image = np.array(img_pil1)
image.shape

(300, 300, 4)

In [77]:
img_tensor = preprocess(img_pil1)
img_tensor.unsqueeze_(0)
img_tensor = img_tensor[:,:3,:,:]

In [78]:
LABELS_URL = 'https://s3.amazonaws.com/outcome-blog/imagenet/labels.json'
labels = {int(key):value for (key, value)
          in requests.get(LABELS_URL).json().items()}

In [79]:
img_variable = Variable(img_tensor)
fc_out = net(img_variable)

In [104]:
scorelist = fc_out.data.numpy()
args = np.argsort(scorelist).flatten().tolist()
scores = scorelist.flatten().tolist()
args[-10:]

[513, 776, 875, 650, 889, 486, 420, 546, 402, 819]

In [105]:
[(labels[ind], scores[ind]) for ind in args[-10:]]

[('cornet, horn, trumpet, trump', 8.411750793457031),
 ('sax, saxophone', 8.485982894897461),
 ('trombone', 8.974251747131348),
 ('microphone, mike', 9.215702056884766),
 ('violin, fiddle', 9.553831100463867),
 ('cello, violoncello', 9.742064476013184),
 ('banjo', 9.751826286315918),
 ('electric guitar', 9.936294555664062),
 ('acoustic guitar', 11.694685935974121),
 ('stage', 12.047249794006348)]